# Generative Spaces (ABM)

In this workshop we will lwarn how to construct a ABM (Agent Based Model) with spatial behaviours, that is capable of configuring the space. This file is a simplified version of Generative Spatial Agent Based Models. For further information, you can find more advanced versions here:

* [Object Oriented version](https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w3_generative_spaces.ipynb)
* [Vectorized version](https://topogenesis.readthedocs.io/notebooks/random_walker)

## 0. Initialization

### 0.1. Load required libraries

In [1]:
# !pip install ipyvtk_simple

In [2]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import pandas as pd
import numpy as np
np.random.seed(0)

### 0.2. Define the Neighborhood (Stencil)

In [3]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)
print(stencil)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 0 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


### 0.3 Visualize the Stencil

In [4]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = np.array(stencil.shape) + 1
# The bottom left corner of the data set
grid.origin = [0,0,0]
# These are the cell sizes along each axis
grid.spacing = [1,1,1]

# Add the data values to the cell data
grid.cell_arrays["values"] = stencil.flatten(order="F")  # Flatten the stencil
threshed = grid.threshold([0.9, 1.1])

# adding the voxels: light red
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)

# plotting
#p.show(use_ipyvtk=True)

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x0000016B6BE43510) at 0x0000016B73395400>

## 1. Setup the Environment

### 1.1. Load the envelope lattice as the avialbility lattice

In [5]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

### 1.2 Load Program

In [6]:
program_complete1 = pd.read_csv("../data/program_small.csv")
print(program_complete1)

program_complete = pd.read_csv("../data/program_big.csv")
print(program_complete)


    space_name  space_id  str_acc  sun_acc
0        lobby         0      1.0      0.0
1  roof_garden         1      0.0      1.0
2     workshop         2      0.5      0.5
                                        space_name  space_id  str_acc  sun_acc
0                       Student_housing_(1_person)         0      0.4      0.8
1                      Student_housing_(4_persons)         1      0.4      0.8
2                       Assisted_living_(30_units)         2      0.4      0.8
3                      Starter_housing_(100_units)         3      0.4      0.8
4   Parking_spaces_(minimal_of_0.5_per_apartement)         4      1.0      0.2
5                                  Bicycle_parking         5      1.0      0.2
6                                       Vegetation         6      0.0      1.0
7                                        Workshops         7      0.8      0.4
8                                 Co-working_space         8      0.6      0.4
9                                     

In [7]:
program_prefs = program_complete.drop(["space_name","space_id"], 1)
program_prefs

,str_acc,sun_acc
0,0.4,0.8
1,0.4,0.8
2,0.4,0.8
3,0.4,0.8
4,1.0,0.2
5,1.0,0.2
6,0.0,1.0
7,0.8,0.4
8,0.6,0.4
9,0.9,0.4


### 1.2 Load the value fields

In [8]:
# loading the lattice from csv
fields = {}
for f in program_prefs.columns:
    lattice_path = os.path.relpath('../data/' + f + '.csv')
    fields[f] = tg.lattice_from_csv(lattice_path)
fields["str_acc"]

lattice([[[1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 1.  , 1.  , 1.  , 1.  ]],

         [[1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 0.3 , 0.25, 0.2 , 1.  ],
          [1.  , 0.3 , 0.25, 0.2 , 1.  ],
          [1.  , 0.3 , 0.25, 0.2 , 1.  ],
          [1.  , 0.3 , 0.25, 0.2 , 1.  ],
          [1.  , 0.25, 0.2 , 0.15, 1.  ],
          [1.  , 0.2 , 0.15, 0.1 , 1.  ],
          [1.  , 0.15, 0.1 , 0.05, 1.  ],
          [1.  , 0.1 , 0.05, 0.  , 1.  ]],

         [[1.  , 1.  , 1.  , 1.  , 1.  ],
          [1.  , 0.35, 0.3 , 0.25, 1.  ],
          [1.  , 0.35, 0.3 , 0.25, 1.  ],
          [1.  , 0.35, 0.3 , 0.25, 1.  ],
          [1.  , 0.35, 0.3 , 0.25, 1.  ],
          [1.  , 0.3 , 0.25, 0

### 1.3. Initialize the Agents

In [9]:
# initialize the occupation lattice
occ_lattice = avail_lattice * 0 - 1

# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
avail_index = np.array(np.where(avail_lattice == 1)).T

# Randomly choosing three available voxels
agn_num = len(program_complete)
select_id = np.random.choice(len(avail_index), agn_num)
agn_origins = avail_index[select_id]
print(agn_origins)
# adding the origins to the agents locations
agn_locs = []
# for each agent origin ... 
for a_id, a_origin in enumerate(agn_origins):
    
    # add the origin to the list of agent locations
    agn_locs.append([a_origin])

    # set the origin in availablity lattice as 0 (UNavailable)
    avail_lattice[tuple(a_origin)] = 0
    
    # set the origin in occupation lattice as the agent id (a_id)
    occ_lattice[tuple(a_origin)] = a_id



[[10  6  2]
 [ 2  8  3]
 [ 6  4  1]
 [11  5  1]
 [13  8  3]
 [11  6  1]
 [ 1  4  1]
 [12  3  2]
 [15  1  2]
 [13  5  3]
 [15  6  2]
 [ 4  6  1]
 [ 3  8  2]
 [ 4  6  2]
 [11  5  2]
 [ 2  6  1]
 [ 4  6  1]
 [10  7  1]
 [ 4  6  2]
 [10  4  1]]


### 1.4. Visualize the environment

In [14]:
p = pv.Plotter(notebook=True)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(occ_lattice.shape) + 1

# The bottom left corner of the data set
grid.origin = occ_lattice.minbound - occ_lattice.unit * 0.5

# These are the cell sizes along each axis
grid.spacing = occ_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_arrays["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([-0.1, agn_num - 0.9])
# adding the voxels
p.add_mesh(threshed, show_edges=True, opacity=1.0, show_scalar_bar=False)

# adding the availability lattice
init_avail_lattice.fast_vis(p)

#p.show(use_ipyvtk=True)

## 2. ABM Simulation (Agent Based Space Occupation)

### 2.1. Running the simulation

In [15]:
# make a deep copy of occupation lattice
cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
# initialzing the list of frames
frames = [cur_occ_lattice]

# setting the time variable to 0
t = 0
n_frames = 30
# main feedback loop of the simulation (for each time step ...)
while t<n_frames:
    # for each agent ... 
    for a_id, a_prefs in program_complete.iterrows():
        # retrieve the list of the locations of the current agent
        a_locs = agn_locs[a_id]
        # initialize the list of free neighbours
        free_neighs = []
        # for each location of the agent
        for loc in a_locs:
            # retrieve the list of neighbours of the agent based on the stencil
            neighs = avail_lattice.find_neighbours_masked(stencil, loc = loc)
            
            # for each neighbour ... 
            for n in neighs:
                # compute 3D index of neighbour
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                # if the neighbour is available... 
                if avail_lattice[neigh_3d_id]:
                    # add the neighbour to the list of free neighbours
                    free_neighs.append(neigh_3d_id)
        # check if found any free neighbour
        if len(free_neighs)>0:
            # convert free neighbours to a numpy array
            fns = np.array(free_neighs)

            # find the value of neighbours
            # init the agent value array
            a_eval = np.ones(len(fns))
            # for each field...
            for f in program_prefs.columns:
                # find the raw value of free neighbours...
                vals = fields[f][fns[:,0], fns[:,1], fns[:,2]]
                # raise the the raw value to the power of preference weight of the agent
                a_weighted_vals = vals ** a_prefs[f]
                # multiply them to the previous weighted values
                a_eval *= a_weighted_vals

            # select the neighbour with highest evaluation
            selected_int = np.argmax(a_eval)
            # find 3D integer index of selected neighbour
            selected_neigh_3d_id = free_neighs[selected_int]
            # find the location of the newly selected neighbour
            selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()
            # add the newly selected neighbour location to agent locations
            agn_locs[a_id].append(selected_neigh_loc)
            # set the newly selected neighbour as UNavailable (0) in the availability lattice
            avail_lattice[selected_neigh_3d_id] = 0
            # set the newly selected neighbour as OCCUPIED by current agent 
            # (-1 means not-occupied so a_id)
            occ_lattice[selected_neigh_3d_id] = a_id

    # constructing the new lattice
    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
    # adding the new lattice to the list of frames
    frames.append(new_occ_lattice)
    # adding one to the time counter
    t += 1

### 2.2. Visualizing the simulation

In [16]:
p = pv.Plotter(notebook=True)

base_lattice = frames[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.1, agn_num - 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic")
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(247.21077512227743, 152.21077512227743, 232.21077512227743),
 (35.0, -60.0, 20.0),
 (0.0, 0.0, 1.0)]

### 2.3. Saving lattice frames in CSV

In [13]:
for i, lattice in enumerate(frames):
    csv_path = os.path.relpath('../data/abm_animation/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\abm_animation\\abm_f_000.csv'

### Credits

In [ ]:
__author__ = "Shervin Azadi "
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Agent Based Models for Generative Spaces"